# Libraries

In [2]:
import os
import pyspark.sql.functions as F
from pyspark.sql.types import *
import pandas as pd
from gcpspark import *

Dotenv installed
Loaded variables .env True
Pyspark installed
JAVA:  /usr/lib/jvm/temurin-8-jdk-amd64
DATAPROC:  True
Current GCP Project Name: cencosudx
Current ENVIRONMENT: staging
<class 'Exception'>


In [3]:
spark_name = "spark_co_extraction_sales"
spark = create_pyspark(name = spark_name , connection="GCP", verbose=True)

File already exists at: /jars/spark-bigquery-with-dependencies_2.12-0.26.0.jar
File already exists at: /jars/gcs-connector-hadoop3-2.2.19.jar
Process: spark_co_extraction_sales_1746115922854


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/01 16:12:07 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
25/05/01 16:12:07 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
25/05/01 16:12:07 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
25/05/01 16:12:07 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [4]:
table_id = 'cencosudx.data_analytics_default.BQ_CO_SM_SALES_01'

In [5]:
df = spark.read \
    .format("bigquery") \
    .option("table",table_id)\
    .load()

In [6]:
df = df.select(
    F.col("ean_venta").alias("EAN"),
    F.col("sku_venta").alias("Item_Id"),
    F.col("tienda_venta").alias("Location_Id"),
    F.col("fecha_venta").alias("Date"),
    F.col("hora_venta").alias("Hour_Sales"),
    F.col("canal_venta").alias("Sales_Channel"),
    F.col("unidades_venta").alias("Quantity_Sales"),
    F.col("monto_neto_venta").alias("NetAmount"),
    F.col("monto_costo_venta").alias("NetCost")
).distinct()

In [7]:
import pyspark.sql.functions as F
#new_data_1 = df.filter(F.col('fecha_venta') >= '2025-01-01')

In [ ]:
df.write.mode("overwrite").parquet("gs://staging-da-contribution/CO/sales")

#df.write.format("bigquery").option("table","data_analytics_default.BQ_CO_SM_RAW_01")\
#    .option("temporaryGcsBucket","staging-bucket-dataproc-bigquery/dataproc/co_umv").mode("overwrite").save()



In [9]:
print("DONE")

DONE


In [ ]:
df = df.withColumn("Date", F.to_date("Date"))
df_sku_por_mes = df.groupBy(F.year("Date").alias("year"), F.month("Date").alias("month")).agg(F.countDistinct("Item_Id").alias("cant_sku"))
df_sku_por_mes = df_sku_por_mes.orderBy("year", "month")
df_sku_por_mes.show(50) 

In [10]:
spark.stop()

In [11]:
print("ok")

ok
